Libraries and Globals

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from IPython.display import display
from PIL import Image
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

# import warnings
# with warnings.catch_warnings():
#     warnings.simplefilter(action='ignore', category=FutureWarning)

"""Default Global Values for splitting criteria in decision trees"""
MINIMAL_SIZE_FOR_SPLIT = 1

MAXIMUM_DEPTH = 1000

MINIMAL_INFO_GAIN = 0

optimize_numerical = False

random.seed(1)
TREE_NUMS = [1,2,5,10,20,30,50]
NUM_FOLDS = 10

Decision Tree ADT

In [2]:
class Node:
    #building the tree
    def __init__(self, is_numeric):
        self.isLeaf = False
        self.numeric = is_numeric
        
    def set_attribute(self, attribute, split_value):
        self.attribute = attribute
        self.split_value = split_value
        
    def set_as_leaf(self, decision):
        self.isLeaf = True
        self.decision = decision
        
    def set_left(self, node):
        self.left = node
        
    def set_right(self, node):
        self.right = node
        
    #making a decision
    def get_attribute(self):
        return self.attribute
    def get_split_value(self):
        return self.split_value
    def is_numeric(self):
        return self.numeric
    
    def get_left(self):
        return self.left
    def get_right(self):
        return self.right
    
    def is_leaf(self):
        return self.isLeaf
    def get_decision(self):
        return self.decision

Decision Tree mechanics

In [3]:
def compute_entropy(dataset):
    dataset = dataset['class']
    N = dataset.shape[0]
    class_list = dataset.value_counts()

    entropy = 0
    for count in class_list.values:
        prob = count / N
        log_prob = np.log2(prob)
        entropy -= prob * log_prob
    return entropy

def split_dataset(dataset, numerical, attribute, split_val):
    if numerical:
        left = dataset[dataset[attribute] < split_val]
        right = dataset[dataset[attribute] >= split_val]
        return left, right
    else:
        right = dataset.loc[dataset[attribute] == split_val]
        left = dataset.loc[dataset[attribute] != split_val]
        return left, right

def entropy_column(dataset, attribute):
    att_name = attribute.iloc[0]
    data_type = attribute.iloc[1]
    N = dataset.shape[0]
    
    

    if data_type == 'numerical':
        if optimize_numerical:
            split = np.mean(dataset.iloc[:,0])
            
            left, right = split_dataset(dataset, numerical=True, attribute=att_name, split_val=split)
            left_length = left.shape[0]
            right_length = right.shape[0]
            
            left_entropy = compute_entropy(left)
            right_entropy = compute_entropy(right)

            entropy = left_entropy * (left_length/N) + right_entropy * (right_length/N)
            return entropy, split
        
        #this method sorts the column by ascending unique values, takes the average of two consecutive values,
        #   and performs the split on the resulting value. Then the one with the lowest resulting entropy is chosen
        #   and returns both the entropy and split val
        dataset = dataset.sort_values(by=dataset.columns[0])
        
        #gets unique values and creates information tracker
        att_values = dataset.iloc[:, 0].unique()
        split_entropy = np.c_[np.zeros(len(att_values) - 1), np.zeros(len(att_values) - 1)]
        
        
        for i in range(len(att_values)-1):
            split = (att_values[i] + att_values[i+1])/2
            split_entropy[i, 0] = split
            
            #splits dataset above and below split
            left, right = split_dataset(dataset, numerical=True, attribute=att_name, split_val=split)
            
            left_length = left.shape[0]
            right_length = right.shape[0]
            
            left_entropy = compute_entropy(left)
            right_entropy = compute_entropy(right)

            entropy = left_entropy * (left_length/N) + right_entropy * (right_length/N)
            split_entropy[i,1] = entropy
        
        if split_entropy.shape[0] == 0:
            return 0, None
        if split_entropy.shape[0] == 1:
            return split_entropy[0,1], split_entropy[0,0]
        
        min_entropy_index = np.argmin(split_entropy[:,1])
        
        #first item returned is the entropy, second is the value we choose to split at
        return split_entropy[min_entropy_index,1], split_entropy[min_entropy_index, 0]
    
    else:
        assert data_type == 'categorical'
        
        category_values = dataset[att_name].value_counts()
        categories = np.transpose(category_values.axes)
        
        split_entropy = pd.DataFrame(data=np.c_[categories, np.zeros(categories.shape[0])], columns=['category', 'entropy'])
        categories = categories.flatten()
        
        for cat in categories:
            part_one, part_two = split_dataset(dataset, numerical=False, attribute=att_name, split_val=cat)
            
            part_one_entropy = compute_entropy(part_one)
            part_two_entropy = compute_entropy(part_two)
            
            n = category_values[cat] #num times cat appears in dataset
            entropy = part_one_entropy * n / N + part_two_entropy * (N-n) / N #weighed avg
            split_entropy.loc[split_entropy['category'] == cat, 'entropy'] = entropy
        
        min_row = np.argmin(split_entropy['entropy'])
        
        return split_entropy.loc[min_row, 'entropy'], split_entropy.loc[min_row, 'category']

def determine_attribute(dataset, attribute_list):
    attribute_list = attribute_list.copy()
    
    presplit_entropy = compute_entropy(dataset)
    
    #sets up information for debugging and decision
    #attribute_list has columns 'attribute', 'attribute type', 'information gain', and 'split_value
    info_gain = np.ones(attribute_list.shape[0]) * presplit_entropy #stores info_gain
    
    #this column either contains the numerical split value or class to separate
    split_value = np.zeros(attribute_list.shape[0])
    
    attribute_list.insert(2, 'info_gain', info_gain)
    attribute_list.insert(3, 'split_value', split_value)
    attribute_list = attribute_list.drop(attribute_list[attribute_list['attribute'] == 'class'].index)
    attribute_list = attribute_list.reset_index(drop=True)
    
    #computes info gain for each column
    for i in range(len(attribute_list)):
        attribute = attribute_list.loc[i, 'attribute']
        
        column = dataset[[attribute, 'class']]
        
        gain, split_val = entropy_column(column, attribute_list.iloc[i])
        
        attribute_list.loc[i, 'info_gain'] -= gain
        attribute_list.loc[i, 'split_value'] = split_val
    
    #sorts and chooses attribute with highest info gain
    attribute_list = attribute_list.sort_values(by='info_gain', ascending=False)
    return attribute_list.iloc[0]

def get_majority_class(dataset):
    dataset = dataset['class']
    highest_class = dataset.value_counts().index[0]
    return highest_class
    

Decision Tree Code

In [4]:
def decision_tree(dataset, attribute_list, depth=0):
    #stopping criteria:
    curr_entropy = compute_entropy(dataset)
    if (curr_entropy == 0 or attribute_list.shape[0] <= MINIMAL_SIZE_FOR_SPLIT or depth >= MAXIMUM_DEPTH): 
        leaf = Node(False)
        decision = get_majority_class(dataset)
        leaf.set_as_leaf(decision)
        return leaf
    
    #determines best attribute to split on
    split_attribute = determine_attribute(dataset, attribute_list)
    
    #more stopping criteria based on information gain
    if split_attribute['info_gain'] <= MINIMAL_INFO_GAIN: 
        leaf = Node(False)
        decision = get_majority_class(dataset)
        leaf.set_as_leaf(decision)
        return leaf
    
    #splits dataset based on the value of the attribute
    #   for categorical: right node if attribute equals split value else left node
    #   for numerical: right node if attribute greater than or equal to split value else left node
    numerical = True if split_attribute['att_type'] == 'numerical' else False
    
    parent = Node(is_numeric=numerical)
    parent.set_attribute(split_attribute['attribute'], split_attribute['split_value'])
    
    left_dataset, right_dataset = split_dataset(dataset, numerical, 
                                                split_attribute['attribute'], 
                                                split_val=split_attribute['split_value'])
    
    #removes category once we split on it
    if not numerical:
        attribute_name = split_attribute['attribute']
        attribute_list = attribute_list[attribute_list['attribute'] != attribute_name]
        
        parent.set_attribute(split_attribute['attribute'], split_attribute['split_value'])
    
    
    #recursively creates child nodes
    if len(left_dataset) == 0: #set to majority class in dataset if split results in complete decision
        left_node = Node(False)
        decision = get_majority_class(dataset)
        left_node.set_as_leaf(decision)
    else:
        if left_dataset.size == 0:
            print('error')
        left_node = decision_tree(left_dataset, attribute_list, depth=(depth+1))

    if len(right_dataset) == 0:
        right_node = Node(False)
        decision = get_majority_class(dataset)
        right_node.set_as_leaf(decision)
    else:
        right_node = decision_tree(right_dataset, attribute_list, depth=(depth+1))
    
    
    parent.set_left(left_node)
    parent.set_right(right_node)
    
    return parent

def traverse_tree(tree, observation):
    while not tree.is_leaf():
        curr_att = tree.get_attribute()
        is_numeric = tree.is_numeric()
        split_val = tree.get_split_value()
        x = observation[curr_att]
        if is_numeric:
            if x >= split_val:
                tree = tree.get_right()
            else:
                tree = tree.get_left()
        else:
            if x == split_val:
                tree = tree.get_right()
            else:
                tree = tree.get_left()
    decision = tree.get_decision()
    return decision

def predict(tree, dataset):
    #y_hat and y_true
    y_hat = np.empty(dataset.shape[0], dtype='int')
    
    for i in range(dataset.shape[0]):
        observation = dataset.iloc[i]
        y_hat[i] = traverse_tree(tree, observation)
    
    return y_hat

Preparing cross-validation, metrics, and bootstrapping

In [5]:
def stratify_dataset(dataset, folds):
    classes = dataset['class'].value_counts()
    class_info = pd.DataFrame({'class': classes.index, 'count': classes.values})
    classes = list(classes.axes)
    class_info['fold_size'] = np.floor(class_info['count'] / folds)
    class_info['remainder'] = class_info['count'] % folds

    dataset = dataset.sample(frac=1)
    subsets = []
    for fold in range(folds):
        new_fold = pd.DataFrame(columns=dataset.columns)
        for c in class_info['class']:
            fold_size = int(class_info.loc[class_info['class'] == c, 'fold_size'].iloc[0])
            if class_info.loc[class_info['class'] == c, 'remainder'].iloc[0] > 0:
                class_info.loc[class_info['class'] == c, 'remainder'] -= 1
                fold_size += 1
            
            observations = dataset.loc[dataset['class'] == c].head(fold_size)
            new_fold = pd.concat([new_fold, observations], ignore_index=True)
            # new_fold = new_fold.append(observations, ignore_index=True)
            dataset = dataset.drop(observations.index)
        new_fold = new_fold.sample(frac=1)
        subsets.append(new_fold)
    
    return subsets

def make_statistics(output):
    n = output.shape[0]
    
    #makes df counting confusion matrix values:
    classes = list(output['y_true'].value_counts().index)
    num_classes = len(classes)
    
    true_pos = np.zeros(num_classes)
    false_pos = np.zeros(num_classes)
    true_neg = np.zeros(num_classes)
    false_neg = np.zeros(num_classes)
    
    data = {
    'class': classes,
    'true_pos': true_pos,
    'true_neg': true_neg,
    'false_pos': false_pos,
    'false_neg': false_neg
    }
    
    confusion_matrix = pd.DataFrame(data, dtype='int')
    
    #classifies prediction by true and false pos and neg
    for i in range(n):
        y_true = output.loc[i, 'y_true']
        y_hat = output.loc[i, 'y_hat_majority']
        class_row = confusion_matrix.index[confusion_matrix['class'] == y_true][0]
        
        if y_hat == y_true:
            #adds true positive for y_true
            confusion_matrix.loc[class_row, 'true_pos'] += 1
            
            #adds true negatives for every class except y_true
            for j in range(num_classes):
                if j == class_row:
                    continue
                else:
                    confusion_matrix.loc[j, 'true_neg'] += 1
        else: #prediction is different from true class
            
            #adds a false negative for y_true
            confusion_matrix.loc[class_row, 'false_neg'] += 1
            
            #adds false positives for every other class
            for j in range(num_classes):
                if j == class_row:
                    continue
                else:
                    confusion_matrix.loc[j, 'false_pos'] += 1
    
    accuracy = confusion_matrix['true_pos'].sum() / n

    precision = []
    recall = []
    for c in range(num_classes):
        tp = confusion_matrix.loc[c, 'true_pos']
        fp = confusion_matrix.loc[c, 'false_pos']
        fn = confusion_matrix.loc[c, 'false_neg']
        
        precision_c = tp / (tp + fp)
        recall_c = tp / (tp + fn)
        
        precision.append(precision_c)
        recall.append(recall_c)
        
    precision = np.mean(precision)
    recall = np.mean(recall)

    beta = 1
    f1_score = (1 + np.square(beta)) * (precision * recall) / (np.square(beta) * precision + recall)
    
    statistics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1-score': f1_score
    }
    return statistics

def subset_attributes(attribute_list):
    
    #samples ~a^(1/2) atributes (with replacement) to train subtree
    num_attributes = attribute_list.shape[0]
    num_subcolumns = int(np.floor(np.sqrt(num_attributes)) + 1)
    
    attribute_sublist = np.empty(shape=(num_subcolumns,2), dtype='<U100')
    
    for i in range(num_subcolumns):
        r = random.randint(0, (num_attributes - 1)) #inclusive
        
        att = attribute_list.loc[r,'attribute']
        att_type = attribute_list.loc[r,'att_type']
        
        while att == 'class':
            r = random.randint(0, (num_attributes - 1))
            att = attribute_list.loc[r,'attribute']
            att_type = attribute_list.loc[r,'att_type']
        
        attribute_sublist[i] = [att, att_type]
    
    attribute_sublist = pd.DataFrame(data=attribute_sublist, columns=attribute_list.columns)
    
    return attribute_sublist

Compiles Decision trees into forrests

In [6]:
def run_forest(train, test, attribute_list, output, ntree):
    #creates ntree voters which classify the test set
    for t in range(ntree):
        
        #Bootstrap Aggregating:
        
        attribute_sublist = subset_attributes(attribute_list)
        
        tree = decision_tree(train, attribute_sublist)
        
        y_hat = predict(tree, test)
        label = 'y_hat' + str(t)
        output[label] = y_hat
    
    return output
    
    
def random_forest(dataset, attribute_list, num_folds, ntree):
    dataset = dataset.sample(frac=1) #shuffles dataset
    
    subsets = stratify_dataset(dataset, num_folds)

    #prepares output for statistics
    columns = ['y_true', 'y_hat_majority']
    output_total = pd.DataFrame(columns=columns)
    
    #k-folding done here:
    for k in range(num_folds):
        subsets_copy = subsets.copy()
        
        test = subsets_copy.pop(k)

        y_true = test['class']
        output = pd.DataFrame(data=y_true.values,columns=['y_true'])
        
        train = pd.concat(subsets_copy, ignore_index=True)
        
        output = run_forest(train,test, attribute_list, output, ntree)
        
        majority_vote = np.zeros(output.shape[0])
        if ntree > 1:
            majority_vote = [output.iloc[row, 1:ntree].mode().iloc[0] for row in range(output.shape[0])]
        else:
            majority_vote = output['y_hat0']
        
        output = pd.DataFrame(data={'y_true': output['y_true'], 'y_hat_majority': majority_vote})
        output_total = pd.concat([output_total, output], ignore_index=True)
    
    statistics = make_statistics(output_total)
    return statistics

Optimizing the stop condition

In [7]:
def reset_hyperparameters():
    MAXIMUM_DEPTH = 10000
    MINIMAL_SIZE_FOR_SPLIT = 1
    MINIMAL_SIZE_FOR_SPLIT = 1
    return
    
def optimize_stop_condition(dataset, attributes, ntree=5, print_out=False):
    #sorry for this code, i got really lazy here because of a bug somewhere else
    
    #MINIMAL_SIZE_FOR_SPLIT, MAXIMUM_DEPTH, MINIMAL_INFO_GAIN
    #default: 0, really big, 0
    reset_hyperparameters()

    #1:20:2     DEPTH
    depth = np.c_[np.zeros(10),np.zeros(10)]
    ind=0
    for i in range(3,22,2):
        MAXIMUM_DEPTH = i
        out = random_forest(dataset, attributes, num_folds=NUM_FOLDS, ntree=ntree)

        if print_out: print('max-depth ' + str(i) + ': '+ str(out['accuracy']))
        depth[ind,0] = i
        depth[ind,1] = out['accuracy']
        ind+=1
    
    best_depth = depth[np.argmax(depth[:,1]),:]

    MAXIMUM_DEPTH = 10000
    
    #1:200:20   proportion of observations to be eligible to split
    min_size = np.c_[np.zeros(10),np.zeros(10)]
    proportions = [0.01,0.04,0.07,0.10,0.13,0.16,0.19,0.21,0.24,0.27]
    num_obs = dataset.shape[0]
    ind=0
    for p in proportions:
        MINIMAL_SIZE_FOR_SPLIT = p * num_obs
        out = random_forest(dataset, attributes, num_folds=NUM_FOLDS, ntree=ntree)
        
        if print_out: print('min-size ' + str(p) + ': '+ str(out['accuracy']))
        min_size[ind,0] = p
        min_size[ind,1] = out['accuracy']
        ind+=1
    
    best_min_size = min_size[np.argmax(min_size[:,1]),:]
    MINIMAL_SIZE_FOR_SPLIT = 1


    #1%,40%,4%  minimum info gain
    min_gain = np.c_[np.zeros(10),np.zeros(10)]
    ind=0
    proportions = [0.01,0.04,0.07,0.10,0.13,0.16,0.19,0.21,0.24,0.27]
    for p in proportions:
        MINIMAL_INFO_GAIN = p
        out = random_forest(dataset, attributes, num_folds=NUM_FOLDS, ntree=ntree)
        
        if print_out: print('min-gain ' + str(p) + ': '+ str(out['accuracy']))
        min_gain[ind,0] = p
        min_gain[ind,1] = out['accuracy']
        ind+=1
    
    best_min_gain = min_gain[np.argmax(min_gain[:,1]),:]
    MINIMAL_INFO_GAIN = 0
    
    #no stopping condition case:
    out = random_forest(dataset, attributes, num_folds=NUM_FOLDS, ntree=ntree)
    acc = out['accuracy']
    if acc >= best_min_size[1] and acc >= best_depth[1] and acc >= best_min_gain[1]:
        print('NO OTHER STOPPING CONDITION')
        reset_hyperparameters()
        return ''
    
    
    if best_min_size[1] >= best_min_gain[1] and best_min_size[1] >= best_depth[1]:
        print('CHOSEN HYPER-PARAMETER: MINIMUM_SIZE_FOR_SPLIT')
        optimize = best_min_size
        MINIMAL_SIZE_FOR_SPLIT = best_min_size[1]
    elif best_min_gain[1] >= best_min_size[1] and best_min_gain[1] >= best_depth[1]:
        print('CHOSEN HYPER-PARAMETER: MINIMAL_INFO_GAIN')
        optimize = best_min_gain
        MINIMAL_INFO_GAIN = best_min_gain[1]
    else:
        print('CHOSEN HYPER-PARAMETER: MAXIMUM_TREE_DEPTH')
        optimize = best_depth
        MAXIMUM_DEPTH = best_depth[1]
    
    out = {
        'n-tree':ntree,
        'min-size':best_min_size[0],
        'min-size-acc':best_min_size[1],
        'min-gain':best_min_gain[0],
        'min-gain-accuracy':best_min_gain[1],
        'max-depth':best_depth[0],
        'max-depth-accuracy':best_depth[1]
    }
    
    if print_out: print(out)
    
    return optimize

Graphs

In [8]:
def make_graphs(dataset, title=''):
    x = dataset['ntree']
    y = dataset.iloc[:,0:4]

    plots = []
    for col in y.columns:
        plt.plot(x,y[col], color='orange')
        
        peak = dataset['ntree'][np.argmax(dataset[col])]
        plt.axvline(x = peak, color = 'black', linestyle='--', label = 'axvline - full height')
        
        plt.title(title + ' ' + col)
        plt.xlabel('Number of Trees (ntree)')
        plt.ylabel(col)
        
        text = 'peak ' + col + ': ' + str(round(np.max(dataset[col]),4)) + ' at ntree=' + str(peak)
        
        #I had to change this for every dataset by the way
        plt.text(20, 0.95, text, fontsize=10, color='red')
    
        plt.grid()

        save_name = 'plots/' + title + '-' + col + '.png'
        plt.savefig(save_name)

        plt.close()

House Votes dataset

In [9]:
# """IMPORTANT: I deleted the first character: '#' in the csv file to make it run"""
# def house_votes():
#     house_votes = pd.read_csv('datasets/hw3_house_votes_84.csv',comment='#', dtype='int')

#     house_votes_attributes = pd.DataFrame(data=np.c_[house_votes.columns, np.empty(len(house_votes.columns))], columns=['attribute', 'att_type'])

#     for i in range(house_votes_attributes.shape[0]):
#         house_votes_attributes.loc[i, 'att_type'] = 'categorical'


#     optimized_param = optimize_stop_condition(house_votes, house_votes_attributes, ntree=10,print_out=True)
#     print(f'stopping condition: {optimized_param}')
    
#     house_stats=[]
#     for ntree in TREE_NUMS:
        
#         out = random_forest(house_votes, house_votes_attributes, num_folds=NUM_FOLDS, ntree=ntree)
        
#         out['ntree'] = ntree
        
#         print('ntree ' + str(ntree) + ' done')
#         print('ACCURACY: ' + str(out['accuracy']))
        
#         house_stats.append(out)
        
#         print(out)
        
#     reset_hyperparameters()
    
#     house_stats = pd.DataFrame(house_stats)
#     return house_stats

# house_stats = house_votes()
# plots = make_graphs(house_stats, title='House-Votes')

Wine Dataset

In [10]:
# """IMPORTANT: I deleted the first two characters: '# ' in the csv file to make it run"""
# def wine():
#     global optimize_numerical
#     wine = pd.read_csv('datasets/hw3_wine.csv', delim_whitespace=True)

#     wine_attributes = pd.DataFrame(data=np.c_[wine.columns, np.empty(len(wine.columns))], columns=['attribute', 'att_type'])

#     for i in range(wine_attributes.shape[0]):
#         wine_attributes.loc[i, 'att_type'] = 'numerical'

#     optimize_numerical = True
#     optimized_param = optimize_stop_condition(wine, wine_attributes, ntree=5,print_out=True)
#     print(f'stopping condition: {optimized_param}')
#     # optimize_numerical = False

#     wine_stats=[]
#     for ntree in TREE_NUMS:
        
#         out = random_forest(wine, wine_attributes, num_folds=NUM_FOLDS, ntree=ntree)
        
#         out['ntree'] = ntree
        
#         print('ntree ' + str(ntree) + ' done')
#         print('ACCURACY: ' + str(out['accuracy']))
        
#         wine_stats.append(out)
        
#         print(out)
        
#     reset_hyperparameters()
    
#     wine_stats = pd.DataFrame(wine_stats)
#     return wine_stats

# wine_stats = wine()
# make_graphs(wine_stats, title='Wine-dataset')

Cancer Dataset:

In [11]:
# def cancer():
#     global optimize_numerical
#     cancer = pd.read_csv('datasets/hw3_cancer.csv', delim_whitespace=True)
#     cancer = cancer.rename(columns={'Class': 'class'}) #i use lowercase to access

#     cancer_attributes = pd.DataFrame(data=np.c_[cancer.columns, np.empty(len(cancer.columns))], columns=['attribute', 'att_type'])

#     for i in range(cancer_attributes.shape[0]):
#         cancer_attributes.loc[i, 'att_type'] = 'numerical'
        

#     optimize_numerical = True
#     optimized_param = optimize_stop_condition(cancer, cancer_attributes, ntree=5,print_out=True)
#     print(f'stopping condition: {optimized_param}')
#     optimize_numerical = False

#     cancer_stats=[]
#     for ntree in TREE_NUMS:
        
#         out = random_forest(cancer, cancer_attributes, num_folds=NUM_FOLDS, ntree=ntree)
        
#         out['ntree'] = ntree
        
#         print('ntree ' + str(ntree) + ' done')
#         print('ACCURACY: ' + str(out['accuracy']))
        
#         cancer_stats.append(out)
        
#         print(out)
        
#     reset_hyperparameters()
    
#     cancer_stats = pd.DataFrame(cancer_stats)
#     return cancer_stats

# cancer_stats = cancer() #9m 50s
# make_graphs(cancer_stats, 'Cancer-dataset')

In [12]:
# def contraceptive():
#     global optimize_numerical
#     #preparing categories and types
#     contraceptive_attributes = pd.DataFrame(data=np.c_[np.empty(10), np.empty(10)], columns=['attribute', 'att_type'])
#     contraceptive_attributes.iloc[0,0] = 'wife-age'
#     contraceptive_attributes.iloc[0,1] = 'numerical'

#     contraceptive_attributes.iloc[1,0] = 'wife-educ'
#     contraceptive_attributes.iloc[1,1] = 'categorical'

#     contraceptive_attributes.iloc[2,0] = 'husband-educ'
#     contraceptive_attributes.iloc[2,1] = 'categorical'

#     contraceptive_attributes.iloc[3,0] = 'num-children'
#     contraceptive_attributes.iloc[3,1] = 'numerical'

#     contraceptive_attributes.iloc[4,0] = 'wife-religion'
#     contraceptive_attributes.iloc[4,1] = 'categorical'

#     contraceptive_attributes.iloc[5,0] = 'wife-working'
#     contraceptive_attributes.iloc[5,1] = 'categorical'

#     contraceptive_attributes.iloc[6,0] = 'husband-job'
#     contraceptive_attributes.iloc[6,1] = 'categorical'

#     contraceptive_attributes.iloc[7,0] = 'living-index'
#     contraceptive_attributes.iloc[7,1] = 'categorical'

#     contraceptive_attributes.iloc[8,0] = 'media-exposure'
#     contraceptive_attributes.iloc[8,1] = 'categorical'

#     contraceptive_attributes.iloc[9,0] = 'class'
#     contraceptive_attributes.iloc[9,1] = 'categorical'

#     contraceptive = pd.read_csv('datasets/contraceptive+method+choice/cmc.data', 
#                                 delimiter=',', header=None,
#                                 names=contraceptive_attributes.iloc[:,0])
    

#     optimize_numerical = True
#     optimized_param = optimize_stop_condition(contraceptive, contraceptive_attributes, ntree=5,print_out=True)
#     print(f'stopping condition: {optimized_param}')
#     # optimize_numerical = False

#     contraceptive_stats=[]
#     for ntree in TREE_NUMS:
        
#         out = random_forest(contraceptive, contraceptive_attributes, num_folds=NUM_FOLDS, ntree=ntree)
        
#         out['ntree'] = ntree
        
#         print('ntree ' + str(ntree) + ' done')
#         print('ACCURACY: ' + str(out['accuracy']))
        
#         contraceptive_stats.append(out)
        
#         print(out)
        
#     reset_hyperparameters()
    
#     contraceptive_stats = pd.DataFrame(contraceptive_stats)
#     return contraceptive_stats

# contraceptive_stats = contraceptive()
# make_graphs(contraceptive_stats, 'Contraceptives')

In [14]:
loan = pd.read_csv('./datasets/loan.csv')

loan = loan.drop(columns={'Loan_ID'})

loan = loan.rename(columns={'Loan_Status':'class'})

#I have to map every single column to numbers
loan['class'] = loan['class'].map({'Y': 1, 'N': 0})
loan['Gender'] = loan['Gender'].map({'Male': 1, 'Female': 0})
loan['Married'] = loan['Married'].map({'Yes': 1, 'No': 0})
loan['Dependents'] = loan['Dependents'].map({'3+': 3, '2':2, '1':1, '0':0})
loan['Education'] = loan['Education'].map({'Graduate': 1, 'Not Graduate': 0})
loan['Self_Employed'] = loan['Self_Employed'].map({'Yes': 1, 'No': 0})
loan['Property_Area'] = loan['Property_Area'].map({'Urban': 2, 'Semiurban': 1, 'Rural':0})

loan_attributes = pd.DataFrame(data=np.c_[loan.columns, np.empty(12)],
                               columns=['attribute', 'att_type'])



loan_attributes.iloc[0,1] = 'categorical'
loan_attributes.iloc[1,1] = 'categorical'
loan_attributes.iloc[2,1] = 'categorical'
loan_attributes.iloc[3,1] = 'categorical'
loan_attributes.iloc[4,1] = 'categorical'
loan_attributes.iloc[5,1] = 'numerical'
loan_attributes.iloc[6,1] = 'numerical'
loan_attributes.iloc[7,1] = 'numerical'
loan_attributes.iloc[8,1] = 'numerical'
loan_attributes.iloc[9,1] = 'categorical'
loan_attributes.iloc[10,1] = 'categorical'
loan_attributes.iloc[11,1] = 'categorical'



stats = random_forest(loan, loan_attributes, num_folds=10, ntree=1)

optimize_numerical = True
# optimize_numerical = False
optimized_param = optimize_stop_condition(loan, loan_attributes, ntree=5,print_out=True)
print(f'stopping condition: {optimized_param}')

loan_stats=[]
for ntree in TREE_NUMS:

    out = random_forest(loan, loan_attributes, num_folds=NUM_FOLDS, ntree=ntree)

    out['ntree'] = ntree

    print('ntree ' + str(ntree) + ' done')
    print('ACCURACY: ' + str(out['accuracy']))

    loan_stats.append(out)

    print(out)

reset_hyperparameters()
    
loan_stats = pd.DataFrame(loan_stats)
# return loan_stats

# contraceptive_stats = contraceptive()
make_graphs(loan_stats, 'Loan')

max-depth 3: 0.7291666666666666


KeyboardInterrupt: 